In [91]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.style.use('fivethirtyeight')

%matplotlib inline

In [116]:
input_X = np.load("../outputs/10_user_X_Nov_11.npy")
input_y = np.load("../outputs/10_user_y_Nov_11.npy")

# put X in the right format
X = np.array([_ for _ in input_X])

# put y in the right format
def ohe(n):
    vec = np.zeros(num_classes_for_prediction)
    vec[n] = 1
    return vec

y = np.array([ohe(n) for n in input_y])

# constants
vocab_size = 254
embedding_dims = 300
tweet_size = 140
batch_size = 100
num_classes_for_prediction = 10

In [121]:
# Model

# Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[None, tweet_size], name='train_inputs')
train_labels = tf.placeholder(tf.int32, shape=[None, num_classes_for_prediction], name='train_labels')

# Create the embedding matrix
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_dims], -1.0, 1.0), name='embeddings')

# Extract the character embeddings for the tweet
embed =  tf.reshape(tf.nn.embedding_lookup(embeddings, train_inputs), [-1, tweet_size, embedding_dims, 1],
                    name='embed')

# The convolutional part
n_gram = 2
num_filters = 32
W_conv = tf.Variable(tf.truncated_normal([n_gram, embedding_size, 1, num_filters], stddev=0.05), name="W_conv")
b_conv = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b_conv")

conv = tf.nn.conv2d(
        embed,
        W_conv, 
        strides=[1, 1, 1, 1], 
        padding="VALID", 
        name="convolution")

h = tf.nn.tanh(tf.nn.bias_add(conv, b_conv), name="relu")

pooled = tf.nn.max_pool(
        h,
        ksize=[1, h.get_shape()[1].value, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="max_pool")

# reshape and add FC
h_pool_flat = tf.reshape(pooled, [-1, num_filters], name='flatten')

W_fc = tf.Variable(tf.truncated_normal([num_filters, num_classes_for_prediction], stddev=0.05), name="W_fc")
b_fc = tf.Variable(tf.constant(0.1, shape=[num_classes_for_prediction]), name="b_fc")
fc_output = tf.nn.tanh(tf.matmul(h_pool_flat, W_fc) + b_fc, name="fc")

# cross entropy loss function
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=train_labels,
    logits=fc_output,
    dim=-1,
    name='loss'
))

preds = tf.argmax(fc_output, 1, name='predictions')
true_vals = tf.argmax(train_labels, 1, name='true_vals')
acc = tf.reduce_mean(tf.cast(tf.equal(preds, true_vals), tf.float32), name = 'acc')

# SGD optimizer
learning_rate = 0.5
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate, name='optimizer')
train_step = optimizer.minimize(loss, name='train_step')

In [123]:
# Train data
from sklearn.model_selection import KFold

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

kf = KFold(n_splits=5, shuffle=True)

cv_accuracy_val = []
cv_loss_val = []

split_num = 0
for train_index, test_index in kf.split(X):
    
    # reset the network parameters
    sess.run(init)
    
    print("Split: {}".format(split_num))
    split_num += 1
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # around 2 minutes per epoch
    for e in range(epochs):
        print("Epoch: {}".format(e))
        pairs = zip(np.split(X, indices_or_sections=np.arange(0, len(X), batch_size)), 
                                    np.split(y, indices_or_sections=np.arange(0, len(X), batch_size)))
        train_acc = []
        train_loss = []
        for batch_x, batch_y in tqdm(pairs):
            if len(batch_x)==batch_size:
                feed_dict = {train_inputs: batch_x, train_labels: batch_y}
                _, train_l, train_a = sess.run([train_step, loss, acc], feed_dict=feed_dict)
                train_acc.append(train_a)
                train_loss.append(train_l)
        
        val_acc, val_loss = sess.run([acc, loss], feed_dict={train_inputs: X_test, train_labels: y_test})
        
        print("Training accuracy: {}".format(np.mean(train_acc)))
        print("Training loss: {}".format(np.mean(train_loss)))
        print("Validation accuracy: {}".format(val_acc))
        print("Validation loss: {}".format(val_loss))
    
    cv_loss_val.append(val_loss)
    cv_accuracy_val.append(val_accuracy)

Split: 0



0it [00:00, ?it/s]
2it [00:00, 10.90it/s]

Epoch: 0



3it [00:00,  8.59it/s]
4it [00:00,  7.75it/s]
901it [02:19,  6.45it/s]
2it [00:00, 10.41it/s]

Training accuracy: 0.5461957454681396
Training loss: 1.4615281820297241
Validation accuracy: 0.7407160401344299
Validation loss: 1.212755560874939
Epoch: 1


...

Training accuracy: 0.8886095285415649
Training loss: 0.938706636428833
Validation accuracy: 0.8790793418884277
Validation loss: 0.9576267600059509
Epoch: 8


901it [01:52,  7.98it/s]
2it [00:00, 13.32it/s]

Training accuracy: 0.8936595916748047
Training loss: 0.935076117515564
Validation accuracy: 0.8831934332847595
Validation loss: 0.953538715839386
Epoch: 9


901it [01:52,  8.01it/s]


Training accuracy: 0.8970523476600647
Training loss: 0.9334972500801086
Validation accuracy: 0.8887529969215393
Validation loss: 0.9509967565536499


In [ ]:
save_dir = "../models/"
model_name = ""
saver = tf.train.Saver()
saver.save(sess, save_dir+model_name)

In [127]:
print("CV Accuracy: {}".format(np.mean(cv_loss_val)))
print("CV Loss: {}".format(np.mean(cv_accuracy_val)))

CV Accuracy: 0.8890116810798645
CV Loss: 0.9492925405502319
